# Multi-Agent Simulation

In [ ]:
%pip install llama-index-program-openai -q

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import nest_asyncio

nest_asyncio.apply()

## Setup Multi-Agent System

In [ ]:
from agentfile.launchers.local import LocalLauncher
from agentfile.agent_server.fastapi import FastAPIAgentServer
from agentfile.control_plane.fastapi import FastAPIControlPlane
from agentfile.message_queues.simple import SimpleMessageQueue

from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.tools import FunctionTool
from llama_index.llms.openai import OpenAI

In [ ]:
import asyncio
import time
import numpy as np


# create agents
def get_the_secret_fact() -> str:
    """Returns the secret fact."""
    time.sleep(2)
    return "The secret fact is: A baby llama is called a 'Cria'."


# create agents
async def aget_the_secret_fact() -> str:
    """Returns the secret fact."""
    await asyncio.sleep(2)
    return "The secret fact is: A baby llama is called a 'Cria'."


def cant_get_secret_fact() -> str:
    """Doesn't return secret fact."""
    time.sleep(8)
    return "I don't know much, tbh."


async def acant_get_secret_fact() -> str:
    """Doesn't return secret fact."""
    await asyncio.sleep(8)
    return "I don't know much, tbh."


secret_tool = FunctionTool.from_defaults(
    fn=get_the_secret_fact, async_fn=aget_the_secret_fact
)
dumb_tool = FunctionTool.from_defaults(
    fn=cant_get_secret_fact, async_fn=acant_get_secret_fact
)

agent1 = FunctionCallingAgentWorker.from_tools([secret_tool], llm=OpenAI()).as_agent()
agent2 = FunctionCallingAgentWorker.from_tools([dumb_tool], llm=OpenAI()).as_agent()

message_queue = SimpleMessageQueue()
control_plane = FastAPIControlPlane(message_queue=message_queue)
agent_server_1 = FastAPIAgentServer(
    agent1,
    message_queue,
    description="Knows what the secret fact is.",
    agent_id="secret_fact_agent",
)
agent_server_2 = FastAPIAgentServer(
    agent2,
    message_queue,
    description="Doesn't know what the secret fact is.",
    agent_id="dumb_fact_agent",
)
# # launch it
# launcher = LocalLauncher([agent_server_1, agent_server_2], control_plane, message_queue)
# launcher.launch_single("What is the secret fact?")

In [ ]:
# Manual Launch
launcher = LocalLauncher([agent_server_1, agent_server_2], control_plane, message_queue)

# register each agent to the control plane
for agent_server in launcher.agent_servers:
    await launcher.control_plane.register_agent(agent_server.agent_definition)

# start agents
for agent_server in launcher.agent_servers:
    asyncio.create_task(agent_server.start_processing_loop())

# runs until the message queue is stopped by the human consumer
mq_task = asyncio.create_task(launcher.message_queue.start())

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:agentfile.message_queues.base:Publishing message: id_='b38b83a8-79ac-41f9-abb6-ebce8c2d3efa' data={'input': 'What is the secret fact?', 'task_id': 'c11269d4-acaf-4289-b54d-e113d78b7d55', 'agent_id': None} action=<ActionTypes.NEW_TASK: 'new_task'> type='secret_fact_agent'
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:agentfile.message_queues.base:Publishing message: id_='e23bbbac-2dc8-467a-be62-5799520b1108' data={'task_id': 'c11269d4-acaf-4289-b54d-e113d78b7d55', 'history': [{'role': <MessageRole.USER: 

Got response:
 {'result': "The secret fact is: A baby llama is called a 'Cria'."}


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:agentfile.message_queues.base:Publishing message: id_='ae38164f-4957-489d-af2f-70c177186ea8' data={'input': 'What is the secret fact?', 'task_id': 'a3c92fc8-6ef2-4ed8-96f3-93f9d3914326', 'agent_id': None} action=<ActionTypes.NEW_TASK: 'new_task'> type='secret_fact_agent'
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:agentfile.message_queues.base:Publishing message: id_='9bfab017-688a-4bd3-b974-0328c0596505' data={'task_id': 'a3c92fc8-6ef2-4ed8-96f3-93f9d3914326', 'history': [{'role': <MessageRole.USER: 'user'>, 'content': 'What is the secret fact?', 'additional_kwargs': {}}, {'role': <MessageRole.ASSISTANT: 'assistant'>, 'content': None, 'additional_kwargs': {'tool_call

Got response:
 {'result': "The secret fact is: A baby llama is called a 'Cria'."}


## Simulation: Incoming Agent Tasks

In [ ]:
import asyncio
import numpy as np
from contextvars import ContextVar
from datetime import datetime

In [ ]:
from agentfile.launchers.local import HumanMessageConsumer
from agentfile.types import ActionTypes, TaskDefinition
from agentfile.messages.base import QueueMessage

# register human consumer
human_consumer = HumanMessageConsumer(
    message_handler={
        ActionTypes.COMPLETED_TASK: launcher.handle_human_message,
    }
)
await launcher.register_consumers([human_consumer])

In [ ]:
async def simulate_incoming_tasks(max_tasks=10):
    task_counter: ContextVar[int] = ContextVar("task_counter", default=0)
    while True:
        # simulate arrival of task
        interarrival_time = np.random.exponential(5)
        await asyncio.sleep(interarrival_time)
        print("New Task Arrival")

        # publish initial task
        data = TaskDefinition(input="What is the secret fact?").dict()
        data.update({"arrival_time": datetime.now()})
        await launcher.message_queue.publish(
            QueueMessage(
                type="control_plane",
                action=ActionTypes.NEW_TASK,
                data=data,
            )
        )

        task_counter_value = task_counter.get()
        updated_task_counter = task_counter_value + 1
        task_counter.set(updated_task_counter)
        if updated_task_counter == max_tasks:
            break

In [ ]:
await asyncio.create_task(simulate_incoming_tasks(2))

INFO:agentfile.message_queues.base:Publishing message: id_='f0f48c4f-a904-438c-958c-38ffdcb19a0e' data={'input': 'What is the secret fact?', 'task_id': 'c11269d4-acaf-4289-b54d-e113d78b7d55', 'agent_id': None, 'arrival_time': datetime.datetime(2024, 6, 9, 18, 31, 32, 516284)} action=<ActionTypes.NEW_TASK: 'new_task'> type='control_plane'


New Task Arrival


INFO:agentfile.message_queues.base:Publishing message: id_='6d3291a6-6688-4d18-93f1-bdc43651b447' data={'input': 'What is the secret fact?', 'task_id': 'a3c92fc8-6ef2-4ed8-96f3-93f9d3914326', 'agent_id': None, 'arrival_time': datetime.datetime(2024, 6, 9, 18, 31, 43, 166292)} action=<ActionTypes.NEW_TASK: 'new_task'> type='control_plane'


New Task Arrival


In [ ]:
launcher.message_queue.queues

{'secret_fact_agent': deque([]),
 'dumb_fact_agent': deque([]),
 'human': deque([]),
 'control_plane': deque([])}

### System Metrics

In [ ]:
import pandas as pd

message_data = {
    "message_id": [],
    "message_type": [],
    "flow_times": [],
    "service_times": [],
    "wait_times": [],
    "arrival_times": [],
    "service_start_times": [],
    "departure_times": [],
}

for agent_consumer in agent_consumers:
    message_data["message_id"] += [m.id_ for m in agent_consumer.processed_messages]
    message_data["message_type"] += [m.type for m in agent_consumer.processed_messages]
    message_data["flow_times"] += [
        m.data["flow_time"] for m in agent_consumer.processed_messages
    ]
    message_data["service_times"] += [
        m.data["service_time"] for m in agent_consumer.processed_messages
    ]
    message_data["wait_times"] += [
        m.data["wait_time"] for m in agent_consumer.processed_messages
    ]
    message_data["arrival_times"] += [
        m.data["arrival_time"] for m in agent_consumer.processed_messages
    ]
    message_data["service_start_times"] += [
        m.data["service_start_time"] for m in agent_consumer.processed_messages
    ]
    message_data["departure_times"] += [
        m.data["departure_time"] for m in agent_consumer.processed_messages
    ]

system_data = pd.DataFrame(message_data)

In [ ]:
system_data

,message_id,message_type,flow_times,service_times,wait_times,arrival_times,service_start_times,departure_times
0,fe6eb649-e5f5-47fa-b829-0350fceec837,A,4.069600,4.002927,0.066673,2024-06-08 17:07:28.252065,2024-06-08 17:07:28.318738,2024-06-08 17:07:32.321665
1,ad32de58-5c9f-4e10-9628-04c66a91ce69,A,14.920089,4.227754,10.692335,2024-06-08 17:07:33.145271,2024-06-08 17:07:43.837606,2024-06-08 17:07:48.065360
2,9bd32a4a-cffc-4c05-a4e7-9e492b04bb8e,A,14.891336,0.824007,14.067329,2024-06-08 17:07:37.471043,2024-06-08 17:07:51.538372,2024-06-08 17:07:52.362379
3,889e106b-e43e-4d49-831d-6d9f03561c27,B,11.414479,11.414194,0.000285,2024-06-08 17:07:32.321731,2024-06-08 17:07:32.322016,2024-06-08 17:07:43.736210
4,f5e646a6-1249-41c9-856e-e7cb2e6e3111,B,3.372360,3.371525,0.000835,2024-06-08 17:07:48.065428,2024-06-08 17:07:48.066263,2024-06-08 17:07:51.437788
5,9b8315f9-f9d7-4d9e-a994-133512aaa583,B,7.626466,7.625623,0.000843,2024-06-08 17:07:52.362445,2024-06-08 17:07:52.363288,2024-06-08 17:07:59.988911


In [ ]:
metric_cols = ["flow_times", "service_times", "wait_times"]
system_data[metric_cols].mean()

flow_times       4.836974
service_times    3.176480
wait_times       1.660494
dtype: float64

In [ ]:
system_data.groupby("message_type")[metric_cols].mean()

,flow_times,service_times,wait_times
message_type,,,
A,6.311056,2.991006,3.320050
B,3.362892,3.361955,0.000937


In [ ]:
agent_consumers[1]

AgentConsumer(id_='3bc4380e-fb99-4698-b8f2-ddf33b866831', message_type='B', processed_messages=[QueueMessage(id_='cead17d5-df84-44cd-9442-4db99d94defc', data={'arrival_time': datetime.datetime(2024, 6, 8, 16, 58, 26, 258449), 'parent_A': '7b4a236c-8da0-4952-b5f8-e1eb4a51c62b', 'service_start_time': datetime.datetime(2024, 6, 8, 16, 58, 26, 258875), 'wait_time': 0.000426, 'departure_time': datetime.datetime(2024, 6, 8, 16, 58, 28, 224559), 'service_time': 1.965684, 'flow_time': 1.96611}, action=None, type='B'), QueueMessage(id_='ffeef176-e175-4e36-b171-5934a2c7c209', data={'arrival_time': datetime.datetime(2024, 6, 8, 16, 58, 29, 417147), 'parent_A': 'eab252fa-e77b-48df-9f47-6a10880a21ae', 'service_start_time': datetime.datetime(2024, 6, 8, 16, 58, 29, 418071), 'wait_time': 0.000924, 'departure_time': datetime.datetime(2024, 6, 8, 16, 58, 50, 322760), 'service_time': 20.904689, 'flow_time': 20.905613}, action=None, type='B'), QueueMessage(id_='8d6f9306-64ae-457d-bea9-a54e90d905ed', data